<a href="https://colab.research.google.com/github/AROY13-PRAETORIAN/StyleGAN_2_ADA/blob/main/StyleGAN2_ADA_AROY13.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## STYLEGAN_2_ADA WITH CUSTOM **DATASET** 

In [ ]:
%tensorflow_version 1.x
from numpy import random
!nvidia-smi
import os
from google.colab import drive
from pathlib import Path
content_path = Path('/').absolute() / 'content'
drive_path = content_path / 'drive'
drive.mount(str(drive_path))

TensorFlow 1.x selected.
Mon Sep 27 05:02:47 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.63.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P8    26W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                               

In [ ]:
#create directory
if os.path.isdir("/content/drive/My Drive/style_gan2_ada"):
    %cd "/content/drive/My Drive/style_gan2_ada/stylegan2-ada"
else:
    #installing stylegan2-ada
    %cd "/content/drive/My Drive/"
    !mkdir style_gan2_ada
    %cd style_gan2_ada
    !git clone https://github.com/dvschultz/stylegan2-ada
    %cd stylegan2-ada
    !mkdir downloads
    !mkdir datasets

/content/drive/My Drive/style_gan2_ada/stylegan2-ada


In [ ]:
%cd "/content/drive/My Drive/style_gan2_ada/stylegan2-ada"
!git config --global user.name "AROY13-PRAETORIAN"
!git config --global user.email "abhigyan.roy2019@vitstudent.ac.in"
!git fetch origin
!git checkout origin/main -- train.py

/content/drive/My Drive/style_gan2_ada/stylegan2-ada


# **DATASET CONVERSION TO .tfrecords**

In [ ]:
#upload to drive and give its path to zip_path
zip_path = "/content/drive/MyDrive/Datasets/resized.zip"
!unzip {zip_path} -d /content/

In [ ]:
dataset_path = "/content/resized" #the dataset.zip will be unzipped in runtime so give path to that folder
dataset_name = "resized" #dataset name give same as folder unzipped in runtime

!python dataset_tool.py create_from_images ./datasets/{dataset_name} {dataset_path}

# **TRAINING**

In [ ]:
!python train.py --help

In [ ]:
#same as the dataset name given earlier
dataset_name = "resized"
#how often samples and .pkl fines are generaed are determined by this
snapshot_count = 2
#mirror images left to right
mirrored = True
#mirror images top to bottom
mirroredY = False
#metric
metric_list = None
#augments
augs = "bgc"
#if you are running it for the first time give the path to ffhq(+resolution)
#if you want to use transfer learning then use a pretrained model uploaded to drive and give path to it or if you are resuming training give path to last .pkl file created in directory
resume_from = ""
!python train.py --outdir ./results --snap={snapshot_count} --cfg=11gb-gpu --data=./datasets/{dataset_name} --augpipe={augs} --mirror={mirrored} --mirrory={mirroredY} --metrics={metric_list} --resume={resume_from} --augpipe="bgc"

# **GENERATE IMAGES**

In [ ]:
%pip install opensimplex
output_path = drive_path / 'MyDrive' / 'stylegan2_ada_output'
output_path_images = output_path / 'images'
stylegan2_main_path = drive_path / 'MyDrive' / 'style_gan2_ada' / 'stylegan2-ada'
if not output_path.is_dir():
    %mkdir "{output_path}"
output_path_images = output_path / 'images'
if not output_path_images.is_dir():
    %mkdir "{output_path_images}"

In [ ]:
seed_init = random.randint(10000)
nbr_images = 100

generation_from = '/content/drive/MyDrive/colab-sg2-ada/stylegan2-ada/results/00005-abstract_art_512-mirror-11gb-gpu-bg-resumecustom/network-snapshot-000100.pkl' # path to final trained model

!python "{stylegan2_main_path / 'generate.py'}" generate-images \
    --outdir="{output_path_images}" --trunc=0.7 \
    --seeds={seed_init}-{seed_init+nbr_images-1} --create-grid \
    --network={generation_from}

Loading networks from "/content/drive/MyDrive/colab-sg2-ada/stylegan2-ada/results/00005-abstract_art_512-mirror-11gb-gpu-bg-resumecustom/network-snapshot-000100.pkl"...
Setting up TensorFlow plugin "fused_bias_act.cu": Loading... Done.
Setting up TensorFlow plugin "upfirdn_2d.cu": Loading... Done.
Generating image for seed 8145 (0/100) ...
Generating image for seed 8146 (1/100) ...
Generating image for seed 8147 (2/100) ...
Generating image for seed 8148 (3/100) ...
Generating image for seed 8149 (4/100) ...
Generating image for seed 8150 (5/100) ...
Generating image for seed 8151 (6/100) ...
Generating image for seed 8152 (7/100) ...
Generating image for seed 8153 (8/100) ...
Generating image for seed 8154 (9/100) ...
Generating image for seed 8155 (10/100) ...
Generating image for seed 8156 (11/100) ...
Generating image for seed 8157 (12/100) ...
Generating image for seed 8158 (13/100) ...
Generating image for seed 8159 (14/100) ...
Generating image for seed 8160 (15/100) ...
Generat

# **LATENT SPACE EXPLORATION**

In [ ]:
walk_types = ['line', 'sphere', 'noiseloop', 'circularloop']
latent_walk_output = output_path
explored_network =  "/content/drive/MyDrive/trained_model/network-snapshot-000056.pkl"  # path to final trained model

seeds = [random.randint(20000) for i in range(20)]
print(','.join(map(str, seeds)))
print("Base seeds:", seeds)
!python "{stylegan2_main_path / 'generate.py'}" generate-latent-walk --network="{explored_network}" \
    --outdir="{latent_walk_output}" --trunc=0.7 --walk-type="{walk_types[2]}" \
    --seeds={','.join(map(str, seeds))} --frames {len(seeds)*20} #generates 400 frames

2364,6250,8263,1696,17376,9310,1605,7334,9942,12535,2082,4718,12747,17829,4336,14306,5000,14619,15939,6491
Base seeds: [2364, 6250, 8263, 1696, 17376, 9310, 1605, 7334, 9942, 12535, 2082, 4718, 12747, 17829, 4336, 14306, 5000, 14619, 15939, 6491]
Loading networks from "/content/drive/MyDrive/trained_model/network-snapshot-000056.pkl"...
Setting up TensorFlow plugin "fused_bias_act.cu": Loading... Done.
Setting up TensorFlow plugin "upfirdn_2d.cu": Loading... Done.
Generating image for step 0/400 ...
Generating image for step 1/400 ...
Generating image for step 2/400 ...
Generating image for step 3/400 ...
Generating image for step 4/400 ...
Generating image for step 5/400 ...
Generating image for step 6/400 ...
Generating image for step 7/400 ...
Generating image for step 8/400 ...
Generating image for step 9/400 ...
Generating image for step 10/400 ...
Generating image for step 11/400 ...
Generating image for step 12/400 ...
Generating image for step 13/400 ...
Generating image for st